In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt
import datetime

In [2]:
# Load astrocyte cellids into a list
astro_list = [58293602559, 49297401554, 5101516913] # 1 vasc and 2 astros 
#[68981812481, 69055094691, 69155944265, 87323560258, 4428499694, 58293602559, 49297401554, 5101516913]

len(astro_list)

3

In [3]:
mesh_dir = 'data/c3_cell_meshes/dec/' # or change to your desired folder
seg_source = "precomputed://gs://h01-release/data/20210601/c3"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [4]:
# make a dictionary to hold the mesh file for each cell id in the list

mesh_dictionary = {}
for cellid in astro_list:
    mesh_file = os.path.join(mesh_dir, str(cellid) + '_0.ply')
    mesh_dictionary['cell_' + str(cellid)] = trimesh.load_mesh(mesh_file)

In [5]:
len(mesh_dictionary)

3

In [6]:
astro_opac = 0.5

# Creating mesh actors with opacity and random colors
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        random_color = list(np.random.random(size=3))
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(mesh_dictionary[cell_key], opacity=astro_opac, color=random_color)

locals().update(mesh_actor)

# Creating a camera object and defining the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA67568C60) at 0x000001BA6A723708>

In [7]:
# render with a centroid calculated from global mean of all cells in the view 

astro_opac = 0.5

# Creating mesh actors with opacity and random colors
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        random_color = list(np.random.random(size=3))
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=random_color
        )
        # Append the centroid of the current mesh to the centroids list
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA67569900) at 0x000001BA6A6D8648>

# gradient rendering

## color gradient linear

In [8]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

astro_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, gradient='linear'):
    if gradient == 'linear':
        opacities = 1 - (distances / max_distance)
    elif gradient == 'exponential':
        opacities = np.exp(-distances / max_distance)
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, gradient='linear')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the opacities
        opacity_array = vtk.vtkFloatArray()
        opacity_array.SetName("Opacity")
        for opacity in opacities:
            opacity_array.InsertNextValue(opacity)
        
        # Add the opacity array to the vtkPolyData
        poly_data.GetPointData().SetScalars(opacity_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.ScalarVisibilityOn()
        mapper.SetScalarModeToUsePointFieldData()
        mapper.SelectColorArray("Opacity")
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756A5A0) at 0x000001BA6A6D8D68>

In [9]:
# render with a centroid calculated from global mean of all cells in the view 

astro_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, gradient='linear'):
    if gradient == 'linear':
        opacities = 1 - (distances / max_distance)
    elif gradient == 'exponential':
        opacities = np.exp(-distances / max_distance)
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, gradient='linear')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the opacities
        opacity_array = vtk.vtkFloatArray()
        opacity_array.SetName("Opacity")
        for opacity in opacities:
            opacity_array.InsertNextValue(opacity)
        
        # Add the opacity array to the vtkPolyData
        poly_data.GetPointData().SetScalars(opacity_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.ScalarVisibilityOn()
        mapper.SetScalarModeToUsePointFieldData()
        mapper.SelectColorArray("Opacity")
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756ABF0) at 0x000001BA6A6D8FA8>

## color gradient exponential

In [10]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

astro_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, gradient='linear'):
    if gradient == 'linear':
        opacities = 1 - (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, gradient='exponential')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the opacities
        opacity_array = vtk.vtkFloatArray()
        opacity_array.SetName("Opacity")
        for opacity in opacities:
            opacity_array.InsertNextValue(opacity)
        
        # Add the opacity array to the vtkPolyData
        poly_data.GetPointData().SetScalars(opacity_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.ScalarVisibilityOn()
        mapper.SetScalarModeToUsePointFieldData()
        mapper.SelectColorArray("Opacity")
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756C530) at 0x000001BA6A7233A8>

In [11]:
# render with a centroid calculated from global mean of all cells in the view 

astro_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, gradient='linear'):
    if gradient == 'linear':
        opacities = 1 - (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, gradient='exponential')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the opacities
        opacity_array = vtk.vtkFloatArray()
        opacity_array.SetName("Opacity")
        for opacity in opacities:
            opacity_array.InsertNextValue(opacity)
        
        # Add the opacity array to the vtkPolyData
        poly_data.GetPointData().SetScalars(opacity_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.ScalarVisibilityOn()
        mapper.SetScalarModeToUsePointFieldData()
        mapper.SelectColorArray("Opacity")
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA67567970) at 0x000001BA6A723DC8>

## opacity gradient

In [12]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

max_opac = 0.2
min_opac = 0.05

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (1 - (distances / max_distance))
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the distances (to be used as scalars)
        distance_array = vtk.vtkFloatArray()
        distance_array.SetName("Distance")
        for distance in distances:
            distance_array.InsertNextValue(distance)
        
        # Add the distance array to the vtkPolyData
        poly_data.GetPointData().SetScalars(distance_array)
        
        # Create a lookup table to map distances to opacities
        lookup_table = vtk.vtkLookupTable()
        lookup_table.SetTableRange(0, max_distance)
        lookup_table.SetNumberOfTableValues(256)
        lookup_table.Build()
        
        for i in range(256):
            value = i / 255.0
            opacity = min_opac + (max_opac - min_opac) * (1 - value)
            color = [1.0, 0.0, 0.0, opacity]  # You can adjust the color as needed
            lookup_table.SetTableValue(i, color)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.SetLookupTable(lookup_table)
        mapper.UseLookupTableScalarRangeOn()
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color (this will be overridden by the lookup table)
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=250)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756B240) at 0x000001BA6A733AC8>

In [13]:
# render with a centroid calculated from global mean of all cells in the view 

max_opac = 0.2
min_opac = 0.05

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (1 - (distances / max_distance))
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the distances (to be used as scalars)
        distance_array = vtk.vtkFloatArray()
        distance_array.SetName("Distance")
        for distance in distances:
            distance_array.InsertNextValue(distance)
        
        # Add the distance array to the vtkPolyData
        poly_data.GetPointData().SetScalars(distance_array)
        
        # Create a lookup table to map distances to opacities
        lookup_table = vtk.vtkLookupTable()
        lookup_table.SetTableRange(0, max_distance)
        lookup_table.SetNumberOfTableValues(256)
        lookup_table.Build()
        
        for i in range(256):
            value = i / 255.0
            opacity = min_opac + (max_opac - min_opac) * (1 - value)
            color = [1.0, 0.0, 0.0, opacity]  # You can adjust the color as needed
            lookup_table.SetTableValue(i, color)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.SetLookupTable(lookup_table)
        mapper.UseLookupTableScalarRangeOn()
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color (this will be overridden by the lookup table)
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756D1D0) at 0x000001BA6A717E28>

## gradient opacity and colormap 

In [14]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
# import matplotlib.pyplot as plt

max_opac = 0.2
min_opac = 0.05

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (1 - (distances / max_distance))
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.Greys # add _r to reverse #hot_r #spring #viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA675723E0) at 0x000001BA6A717EE8>

In [15]:
# render with a centroid calculated from global mean of all cells in the view 

max_opac = 0.2
min_opac = 0.05

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (1 - (distances / max_distance))
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.Greys  # Add _r to reverse #hot_r #spring #viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756E4C0) at 0x000001BA6A717A68>

## reverse opacity gradient

In [16]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
# import matplotlib.pyplot as plt

max_opac = 0.15
min_opac = 0.0005

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756EB10) at 0x000001BA6A7322E8>

In [17]:
# render with a centroid calculated from global mean of all cells in the view 

max_opac = 0.15
min_opac = 0.0005

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA67571D90) at 0x000001BA6A73B6A8>

# refine opacity gradient

In [18]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
# import matplotlib.pyplot as plt

max_opac = 0.9
min_opac = 0  # Slightly higher minimum opacity to ensure visibility

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Debug print to check opacity values
        print(f"Opacities (min: {np.min(opacities)}, max: {np.max(opacities)})")
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.RdBu #PuOr #autumn_r #spring_r #binary #Greys  #hot_r #viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


Opacities (min: 0.02512748457561628, max: 0.8939358477008231)
Opacities (min: 0.08266527085912143, max: 0.8939358477008231)
Opacities (min: 0.008273656194371681, max: 0.8939358477008231)
setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA67570AA0) at 0x000001BA6A6D8CA8>

In [19]:
# render with a centroid calculated from global mean of all cells in the view 

max_opac = 0.9
min_opac = 0  # Slightly higher minimum opacity to ensure visibility

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Debug print to check opacity values
        print(f"Opacities (min: {np.min(opacities)}, max: {np.max(opacities)})")
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.RdBu  # Adjust the colormap as needed
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=500)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


Opacities (min: 0.02512748457561628, max: 0.8939358477008231)
Opacities (min: 0.08266527085912143, max: 0.8939358477008231)
Opacities (min: 0.008273656194371681, max: 0.8939358477008231)
setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA6756FE00) at 0x000001BA6A732FA8>

In [20]:
# add x, y, and z rotation values for the initial view

import numpy as np
import vtk

max_opac = 0.9
min_opac = 0  # Slightly higher minimum opacity to ensure visibility

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        centroids.append(centroid)  # Append centroid to the list

        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Debug print to check opacity values
        print(f"Opacities (min: {np.min(opacities)}, max: {np.max(opacities)})")
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.RdBu  # Adjust the colormap as needed
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Create a camera object and define the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# User input for rotation angles
x_rotation = float(input("Enter rotation angle around the x-axis (in degrees): "))
y_rotation = float(input("Enter rotation angle around the y-axis (in degrees): "))
z_rotation = float(input("Enter rotation angle around the z-axis (in degrees): "))

# Apply rotations to the camera
camera.Azimuth(x_rotation)  # Rotate around the x-axis
camera.Elevation(y_rotation)  # Rotate around the y-axis
camera.Roll(z_rotation)  # Rotate around the z-axis

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


Opacities (min: 0.02512748457561628, max: 0.8939358477008231)
Opacities (min: 0.08266527085912143, max: 0.8939358477008231)
Opacities (min: 0.008273656194371681, max: 0.8939358477008231)
Enter rotation angle around the x-axis (in degrees): 45
Enter rotation angle around the y-axis (in degrees): 0
Enter rotation angle around the z-axis (in degrees): 0
setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA67573080) at 0x000001BA6A7334C8>

In [21]:
# save to disk

extension = 'png'
save_dir = 'vtk_images/'

# Check the length of astro_list to determine the base_name
if len(astro_list) == 1:
    base_name = f"astro_{astro_list[0]}"
else:
    base_name = f"astros_{'_'.join(map(str, astro_list))}"  # Ensure astro_list elements are strings

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# # Create the directory if it doesn't exist
# os.makedirs(save_dir, exist_ok=True)

# # Save the image using the camera from the previous step
# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera  # Use the camera variable defined in the previous step
#                          )


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001BA675710F0) at 0x000001BA0AF79648>